<a href="https://colab.research.google.com/github/rajiv1977/AI/blob/main/CNNTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# TEST DEV
```



In [1]:
# Mount your Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import numpy as np
import cv2
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

In [3]:
class ImageProcessingSettings:
    """Struct to hold settings for image processing."""

    def __init__(self, convert_to_gray=False, resize=False, resize_width=1, resize_height=1, rotate=False, num_classes=10):
        """
        Initialize settings for image processing.

        Args:
            convert_to_gray (bool): Flag to convert images to grayscale.
            resize (bool): Flag to indicate if resizing is needed.
            resize_width (int): Downscale factor for width.
            resize_height (int): Downscale factor for height.
            rotate (bool): Flag to indicate if rotation is needed.
            num_classes (int): Number of classes or features.
        """
        self.convert_to_gray = convert_to_gray
        self.resize = resize
        self.resize_width = resize_width  # New attribute for resize width
        self.resize_height = resize_height  # New attribute for resize height
        self.rotate = rotate
        self.num_classes = num_classes

In [4]:
def load_images_from_folder(folder, settings):
    """
    Load images from specified folder and its subfolders.

    Args:
        folder (str): Path to the main dataset folder.
        settings (ImageProcessingSettings): Settings for image processing.

    Returns:
        np.array: Array of loaded images.
        np.array: Array of labels corresponding to the images.
        list: List of label names.
    """
    images = []
    labels = []
    label_names = []

    # Get subfolder names (class labels)
    subfolders = [f for f in os.listdir(folder) if os.path.isdir(os.path.join(folder, f))]

    # Check if any subfolders were found
    if not subfolders:
        print("Error: No subfolders found in the dataset path.")
        return np.array([]), np.array([]), []

    label_names = subfolders
    num_classes = len(label_names)

    # Iterate through subfolders and load images
    for label_index, subfolder in enumerate(subfolders):
        subfolder_path = os.path.join(folder, subfolder)
        for filename in os.listdir(subfolder_path):
            img_path = os.path.join(subfolder_path, filename)
            try:
                # Load image with appropriate color mode
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE if settings.convert_to_gray else cv2.IMREAD_COLOR)

                # Check if image loading was successful
                if img is None:
                    print(f"Warning: Could not load image: {img_path}")
                    continue

                # Resize image if specified
                if settings.resize:
                    img = cv2.resize(img, (settings.resize_width, settings.resize_height))

                # Add channel dimension for grayscale images if needed
                if settings.convert_to_gray and len(img.shape) == 2:
                    img = img.reshape(img.shape[0], img.shape[1], 1)

                images.append(img)
                labels.append(label_index)
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")

    return np.array(images), np.array(labels), label_names


In [5]:
def build_cnn_model(input_shape, num_classes):
    """
    Build a Convolutional Neural Network (CNN) model using Keras.

    Args:
        input_shape (tuple): Shape of the input images.
        num_classes (int): Number of output classes.

    Returns:
        keras.Model: Compiled CNN model.
    """
    model = models.Sequential()

    # Input layer: (height, width, channels)
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same')) # reduced kernel size, added padding
    model.add(layers.Dropout(0.5))
    model.add(layers.AveragePooling2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same')) # reduced kernel size, added padding
    model.add(layers.Dropout(0.5))
    model.add(layers.AveragePooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())  # Flatten layer
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(0.5))

    # Output layer
    model.add(layers.Dense(num_classes, activation='softmax'))  # Use softmax for multi-class classification

    return model

In [6]:
def main():
    # Parameters
    epochs = 25
    batch_size = 32
    learning_rate = 0.01  # Not used in this example but included for completeness

    # Set the dataset path (update this to your dataset path)
    dataset_path = "/content/drive/My Drive/AI_Outputs/TrainAndValidateDataBackUp"

    # Verify if the dataset path is valid
    if not os.path.isdir(dataset_path):
        print(f"Error: Invalid dataset path: {dataset_path}")
        print("Please ensure the path is correct and the directory exists.")
        return  # Exit the program if the path is invalid

    # Get a list of subfolders and count them
    subfolders = [f for f in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, f))]
    num_classes = len(subfolders)
    print(f"Found {num_classes} subfolders in the dataset path.")

    # Create image processing settings
    settings = ImageProcessingSettings(
        convert_to_gray=True,
        resize=True,
        resize_width=40,
        resize_height=40,
        num_classes=num_classes
    )

    # Load images and labels
    images, labels, label_names = load_images_from_folder(dataset_path, settings)
    labels = to_categorical(labels, settings.num_classes)

    print("Number of images loaded:", len(images))
    print("Shape of first image:", images[0].shape)

    # Split into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.3, random_state=1)  # Changed test_size to 0.3

    # Print the shapes of X_train and y_train
    print("X_train shape:", X_train.shape)
    print("y_train shape:", y_train.shape)

    # Get the actual input shape from the loaded images
    input_shape = images[0].shape

    # Build the model, now passing the actual input shape
    model = build_cnn_model(input_shape, settings.num_classes)

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))

    # Validate the model
    val_loss, val_accuracy = model.evaluate(X_val, y_val)
    print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

if __name__ == "__main__":
    main()

Found 10 subfolders in the dataset path.
Number of images loaded: 8789
Shape of first image: (40, 40, 1)
X_train shape: (6152, 40, 40, 1)
y_train shape: (6152, 10)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
193/193 ━━━━━━━━━━━━━━━━━━━━ 47s 233ms/step - accuracy: 0.1644 - loss: 20.9493 - val_accuracy: 0.3644 - val_loss: 1.9488
Epoch 2/50
193/193 ━━━━━━━━━━━━━━━━━━━━ 41s 212ms/step - accuracy: 0.3667 - loss: 1.8362 - val_accuracy: 0.5279 - val_loss: 1.5269
Epoch 3/50
193/193 ━━━━━━━━━━━━━━━━━━━━ 80s 200ms/step - accuracy: 0.5014 - loss: 1.4729 - val_accuracy: 0.5965 - val_loss: 1.2855
Epoch 4/50
193/193 ━━━━━━━━━━━━━━━━━━━━ 41s 199ms/step - accuracy: 0.5704 - loss: 1.3042 - val_accuracy: 0.6086 - val_loss: 1.2080
Epoch 5/50
193/193 ━━━━━━━━━━━━━━━━━━━━ 39s 204ms/step - accuracy: 0.6198 - loss: 1.1227 - val_accuracy: 0.6625 - val_loss: 1.0836
Epoch 6/50
193/193 ━━━━━━━━━━━━━━━━━━━━ 45s 223ms/step - accuracy: 0.6616 - loss: 0.9972 - val_accuracy: 0.6769 - val_loss: 1.0075
Epoch 7/50
193/193 ━━━━━━━━━━━━━━━━━━━━ 77s 197ms/step - accuracy: 0.6946 - loss: 0.8871 - val_accuracy: 0.6962 - val_loss: 0.9601
Epoch 8/50
193/193 ━━━━━━━━━━━━━━━━━━━━ 40s 190ms/step - accuracy: 0.7136 - loss: 

KeyboardInterrupt: 